In [1]:
from transformers import (
    AutoTokenizer, AutoModelForMaskedLM, 
)

# import nlpaug.augmenter.char as nac
# import nlpaug.augmenter.word as naw
# import nlpaug.augmenter.sentence as nas
# import nlpaug.flow as nafc
# from nlpaug.util import Action

# from googletrans import Translator
# import translators as ts

import re, math, random, json
from copy import deepcopy
from sklearn.model_selection import StratifiedKFold
import pandas as pd
from tqdm import tqdm
from collections import Counter

from module.preprocess import decorate_form, decorate_acd_pair, decorate_asc_pair, decorate_acd_pair_split, decorate_asc_pair_split

import demoji

# from cleantext import clean
# from pykospacing import Spacing
# from hanspell import spell_checker

/home/ubuntu/anaconda3/envs/jeonghyeon/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Raw Data

In [2]:
# train_json = './dataset//nikluge-sa-2022-train.jsonl'
# train_raw = pd.read_json(train_json, lines=True)
# train_raw.annotation = train_raw.annotation.apply(lambda x: str(x))
# train_raw = train_raw[['sentence_form', 'annotation']]
# dup = train_raw.duplicated(keep=False)
# train_raw[dup]

In [3]:
# dev_json = './dataset/nikluge-sa-2022-dev.jsonl'
# dev_raw = pd.read_json(dev_json, lines=True)
# dev_raw.annotation = dev_raw.annotation.apply(lambda x: str(x))
# dev_raw = dev_raw[['sentence_form', 'annotation']]
# dup = dev_raw.duplicated(keep=False)
# dev_raw[dup]

In [4]:
train_json = './dataset/nikluge-sa-2022-train.jsonl'
dev_json = './dataset/nikluge-sa-2022-dev.jsonl'
test_json = './dataset/nikluge-sa-2022-test.jsonl'

In [5]:
train = pd.read_json(train_json, lines=True)
dev = pd.read_json(dev_json, lines=True)
test = pd.read_json(test_json, lines=True)

In [6]:
# train.iloc[2319]
train = train.drop(2319)
# dev.iloc[1692]
dev = dev.drop(1692)

# Declare Stuff to use

In [7]:
### new
entity_property_pair = [
    '본품#가격', '본품#다양성', '본품#디자인', '본품#인지도', '본품#일반', '본품#편의성', '본품#품질',
    '브랜드#가격', '브랜드#디자인', '브랜드#인지도', '브랜드#일반', '브랜드#품질',
    '제품 전체#가격', '제품 전체#다양성', '제품 전체#디자인', '제품 전체#인지도', '제품 전체#일반', '제품 전체#편의성', '제품 전체#품질',
    '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#디자인', '패키지/구성품#일반', '패키지/구성품#편의성', '패키지/구성품#품질'

]
more_tokens = ['&name&', '&affiliation&', '&social-security-num&', '&tel-num&', '&card-num&', '&bank-account&', '&num&', '&online-account&']

tf_id_to_name = ['True', 'False']
tf_name_to_id = {tf_id_to_name[i]: i for i in range(len(tf_id_to_name))}

polarity_id_to_name = ['positive', 'negative', 'neutral']
polarity_name_to_id = {polarity_id_to_name[i]: i for i in range(len(polarity_id_to_name))}

# Count

In [8]:
df = pd.concat([train, dev])
df = pd.concat([train])

count = 0
tags = []
for idx, row in df.iterrows():
    if len(row.annotation) > 0:
        for annotation in row.annotation:
            form = row.sentence_form
            if annotation[1][0] != None and annotation[1][2] > 0:
                tags.append(annotation[0])
                input = []
                start = annotation[1][1]
                end = annotation[1][2]
                if start != 0:
                    input.append(form[:start])
                input.append(form[start:end])
                if len(form) != end:
                    input.append(form[end:])
                count += 1
            else:
                tags.append(annotation[0])
                input = []
                input.append(form)
                count += 1

print(count)

3254


In [9]:
print(len(set(tags)), len(set(entity_property_pair)))
print(set(entity_property_pair)-set(tags))

22 25
{'제품 전체#다양성', '패키지/구성품#가격', '브랜드#디자인'}


In [10]:
tag_counter = Counter(tags)
sorted(tag_counter.items(), key=lambda x: x[1], reverse=True)

[('본품#품질', 1196),
 ('제품 전체#일반', 847),
 ('본품#일반', 256),
 ('제품 전체#품질', 242),
 ('제품 전체#디자인', 158),
 ('본품#편의성', 95),
 ('제품 전체#편의성', 94),
 ('제품 전체#인지도', 66),
 ('패키지/구성품#디자인', 54),
 ('브랜드#일반', 53),
 ('제품 전체#가격', 49),
 ('패키지/구성품#편의성', 38),
 ('패키지/구성품#일반', 30),
 ('본품#다양성', 21),
 ('본품#디자인', 15),
 ('브랜드#품질', 13),
 ('패키지/구성품#품질', 11),
 ('브랜드#인지도', 9),
 ('브랜드#가격', 3),
 ('본품#가격', 2),
 ('패키지/구성품#다양성', 1),
 ('본품#인지도', 1)]

# Change entity_property_pair and Drop rows accordingly

In [11]:
filter = ['본품#품질', 
'제품 전체#일반', 
'본품#일반', 
'제품 전체#품질', 
'제품 전체#디자인', 
'본품#편의성', 
'제품 전체#편의성', 
'제품 전체#인지도', 
'패키지/구성품#디자인', 
'브랜드#일반', 
'제품 전체#가격']

# def remove_props(df):
#     for idx, row in df.iterrows():
#         annotations = []
#         for annotation in row.annotation:
#             # print(annotation)
#             if annotation[0] in filter and annotation[2] == 'positive':
#                 annotations.append(annotation)
#         # print(annotations)
#         row.annotation = annotations
#         # print(row.annotation)
#         # print()
#     df['check'] = df.annotation.apply(lambda x: bool(x))
#     df = df.drop(df[df.check == False].index)
#     return df

def remove_props(df):
    for idx, row in df.iterrows():
        empty = []
        stay = True
        for annotation in row.annotation:
            if annotation[0] not in filter or annotation[2] != 'positive':
                stay = False
        if stay == False:
            row.annotation = empty
    df['check'] = df.annotation.apply(lambda x: bool(x))
    df = df.drop(df[df.check == False].index)
    return df

In [12]:
train = remove_props(train)
dev = remove_props(dev)

In [13]:
dev

,id,sentence_form,annotation,check
0,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,"[[본품#편의성, [부직포 포장, 5, 11], positive]]",True
1,nikluge-sa-2022-dev-00002,목욕할 때마다 넣어봤는데(샤워는 자주 해도 목욕은 그렇게 자주가 아님.. 이것도 약...,"[[본품#일반, [한약풀 냄새, 69, 75], positive]]",True
2,nikluge-sa-2022-dev-00003,혹시 오래된 거 팔지 않나 고민했었는데 쑥향기 자체가 페퍼민트처럼 신선하고 포장도 ...,"[[본품#일반, [쑥향기, 22, 25], positive], [패키지/구성품#디자...",True
3,nikluge-sa-2022-dev-00004,전에는 목욕할 때 입욕제 넣고 하는 거 좋아하지 않았는데 한약재로 바꿔보니 약용까지...,"[[제품 전체#품질, [한약재, 32, 35], positive]]",True
4,nikluge-sa-2022-dev-00005,나 또 시계 좋아하잖아.ㅋㅋ,"[[제품 전체#일반, [시계, 4, 6], positive]]",True
...,...,...,...,...
2786,nikluge-sa-2022-dev-02787,말그대로 좀 순한듯,"[[본품#품질, [None, 0, 0], positive]]",True
2787,nikluge-sa-2022-dev-02788,바르니 촉촉,"[[본품#품질, [None, 0, 0], positive]]",True
2788,nikluge-sa-2022-dev-02789,완전 쫀득도 아니고 그렇다고 너무 물이라서 흐르지 않는 정도를 내가 개인적으로 좋아...,"[[본품#일반, [None, 0, 0], positive]]",True
2792,nikluge-sa-2022-dev-02793,대형으로 샀더니 잘 맞음.,"[[제품 전체#일반, [대형, 0, 2], positive]]",True


# Preprocess

## Cleansing

### Before

In [14]:
# for el in train.sample(n=5).sentence_form:
#     print(el)

In [15]:
# train.sentence_form = train.sentence_form.apply(preprocess)
# dev.sentence_form = dev.sentence_form.apply(preprocess)
# test.sentence_form = test.sentence_form.apply(preprocess)
# total = pd.concat([train, dev])

Test

In [16]:
# case = total.sentence_form.str.contains('r[^A-Za-z0-9가-힣\s]+', case=False, flags=0, na=None, regex=True)
# for e in total[case].sentence_form:
#     print(e)

### After

In [17]:
# for i, row in total[['id', 'sentence_form']].sample(n=5).iterrows():
#     print(row.id, '\t', row.sentence_form)

In [18]:
# total['check'] = total.sentence_form.str.find('OO')
# for row in total[total.check > -1].sentence_form:
#     print(row)
#     break

In [19]:
# total

## Reformat

In [20]:
len(entity_property_pair)

25

In [21]:
decorate_form, decorate_acd_pair, decorate_asc_pair, decorate_acd_pair_split, decorate_asc_pair_split

(<function module.preprocess.decorate_form(form)>,
 <function module.preprocess.decorate_acd_pair(entity)>,
 <function module.preprocess.decorate_asc_pair(entity, sentiment)>,
 <function module.preprocess.decorate_acd_pair_split(entity)>,
 <function module.preprocess.decorate_asc_pair_split(entity, sentiment)>)

In [22]:
def reformat(df):
    ep =[]
    p = []
    for index, row in df.iterrows():
        utterance = row.sentence_form
        id = row.id

        for pair in entity_property_pair:
            isPairInOpinion = False
            if pd.isna(utterance):
                break
            for annotation in row.annotation:
                entity_property = annotation[0]
                sentiment = annotation[2]
                if entity_property not in entity_property_pair:
                    print(row.id, entity_property)
                    break
                if sentiment not in polarity_id_to_name:
                    print(row.id, entity_property, sentiment)
                    break
                if entity_property == pair:
                    form = utterance
                    acd_pair = entity_property
                    # form = decorate_form(utterance)
                    # acd_pair = decorate_acd_pair(entity_property)
                    ep_append = [id, form, acd_pair, tf_name_to_id['True']]
                    ep.append(ep_append)
                    p.append([id, utterance, entity_property, sentiment])
                    isPairInOpinion = True
                    break

            if isPairInOpinion is False:
                form = utterance
                acd_pair = pair
                # form = decorate_form(utterance)
                # acd_pair = decorate_acd_pair(pair)
                ep_append = [id, form, acd_pair, tf_name_to_id['False']]
                ep.append(ep_append)

    return ep, p

In [23]:
def reformat_p_binary(df):
    p_binary = []
    for i, row in df.iterrows():
        row.id, row.form, row.pair, row.sentiment
        for sentiment in polarity_id_to_name:
            if sentiment == row.sentiment:
                form = row.form
                asc_pair = '#'.join([row.pair, row.sentiment])
                # form = decorate_form(row.form)
                # asc_pair = decorate_asc_pair(row.pair, row.sentiment)
                p_binary_append = [row.id, form, asc_pair, tf_name_to_id['True']]
                p_binary.append(p_binary_append)
            else:
                form = row.form
                asc_pair = '#'.join([row.pair, sentiment])
                # form = decorate_form(row.form)
                # asc_pair = decorate_asc_pair(row.pair, sentiment)
                p_binary_append = [row.id, form, asc_pair, tf_name_to_id['False']]
                p_binary.append(p_binary_append)
    return p_binary

In [24]:
len(train), len(dev)

(2716, 2627)

In [25]:
entity_property_pair = [
    '본품#가격', '본품#다양성', '본품#디자인', '본품#인지도', '본품#일반', '본품#편의성', '본품#품질',
    '브랜드#가격', '브랜드#디자인', '브랜드#인지도', '브랜드#일반', '브랜드#품질',
    '제품 전체#가격', '제품 전체#다양성', '제품 전체#디자인', '제품 전체#인지도', '제품 전체#일반', '제품 전체#편의성', '제품 전체#품질',
    '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#디자인', '패키지/구성품#일반', '패키지/구성품#편의성', '패키지/구성품#품질'

]

In [26]:
ep_train, p_train = reformat(train)
ep_train = pd.DataFrame(ep_train, columns=['id', 'form', 'pair', 'labels'])
p_train = pd.DataFrame(p_train, columns=['id', 'form', 'pair', 'sentiment'])

ep_dev, p_dev = reformat(dev)
ep_dev = pd.DataFrame(ep_dev, columns=['id', 'form', 'pair', 'labels'])
p_dev = pd.DataFrame(p_dev, columns=['id', 'form', 'pair', 'sentiment'])

len(ep_train), len(ep_dev), len(p_train), len(p_dev)

(67900, 65675, 2870, 2806)

In [188]:
p_binary_train = reformat_p_binary(p_train)
p_binary_train = pd.DataFrame(p_binary_train, columns=['id', 'form', 'pair', 'labels'])

p_binary_dev = reformat_p_binary(p_dev)
p_binary_dev = pd.DataFrame(p_binary_dev, columns=['id', 'form', 'pair', 'labels'])

In [196]:
len(entity_property_pair)

11

In [189]:
# ep_train.sort_values(['id', 'labels'], inplace=True)
# ep_dev.sort_values(['id', 'labels'], inplace=True)
# p_binary_train.sort_values(['id', 'labels'], inplace=True, ascending=[True, True])
# p_binary_dev.sort_values(['id', 'labels'], inplace=True, ascending=[True, True])

In [192]:
ep_train

,id,form,pair,labels
0,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,제품 전체#인지도,1
1,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,본품#품질,1
2,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,본품#편의성,1
3,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,제품 전체#일반,0
4,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,제품 전체#가격,1
...,...,...,...,...
29871,nikluge-sa-2022-train-03001,일본유니클로 사랑해!!!!!!!,제품 전체#편의성,1
29872,nikluge-sa-2022-train-03001,일본유니클로 사랑해!!!!!!!,브랜드#일반,0
29873,nikluge-sa-2022-train-03001,일본유니클로 사랑해!!!!!!!,패키지/구성품#디자인,1
29874,nikluge-sa-2022-train-03001,일본유니클로 사랑해!!!!!!!,제품 전체#디자인,1


In [193]:
df = p_binary_train
for idx, row in df.iterrows():
    print(row.id, '\n',
          row.form, '\n',
          row.pair, '\n',
          row.labels,  '\n',)
    if idx == 49:
        break

nikluge-sa-2022-train-00003 
 간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다. 
 제품 전체#일반#positive 
 0 

nikluge-sa-2022-train-00003 
 간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다. 
 제품 전체#일반#negative 
 1 

nikluge-sa-2022-train-00003 
 간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다. 
 제품 전체#일반#neutral 
 1 

nikluge-sa-2022-train-00006 
 지금 내 실력과 저질 체력으로는 이 정도 자전거도 되게 훌륭한 거라는.. 
 제품 전체#일반#positive 
 0 

nikluge-sa-2022-train-00006 
 지금 내 실력과 저질 체력으로는 이 정도 자전거도 되게 훌륭한 거라는.. 
 제품 전체#일반#negative 
 1 

nikluge-sa-2022-train-00006 
 지금 내 실력과 저질 체력으로는 이 정도 자전거도 되게 훌륭한 거라는.. 
 제품 전체#일반#neutral 
 1 

nikluge-sa-2022-train-00007 
 내장 기어 3단은 썩 좋은 물건이라 기어 변환도 부드럽고 겉에서는 기어가 보이지 않기 때문에 깔끔하다. 
 본품#품질#positive 
 0 

nikluge-sa-2022-train-00007 
 내장 기어 3단은 썩 좋은 물건이라 기어 변환도 부드럽고 겉에서는 기어가 보이지 않기 때문에 깔끔하다. 
 본품#품질#negative 
 1 

nikluge-sa-2022-train-00007 
 내장 기어 3단은 썩 좋은 물건이라 기어 변환도 부드럽고 겉에서는 기어가 보이지 않기 때문에 깔끔하다. 
 본품#품질#neutral 
 1 

nikluge-sa-2022-train-00011 
 다른 부가 기능은 참 훌륭한데.. 
 본품#품질#positive 
 0 

nikluge-sa-2022-train-00011 

In [194]:
len(ep_train), len(ep_dev), len(p_binary_train), len(p_binary_dev)

(29876, 28897, 8610, 8418)

### save

In [27]:
save_path = './dataset/uncleaned_v4'
!mkdir -p {save_path}

train.to_csv(f'{save_path}/raw_train.csv', index=False)
dev.to_csv(f'{save_path}/raw_dev.csv', index=False)
test.to_csv(f'{save_path}/raw_test.csv', index=False)

ep_train.to_csv(f'{save_path}/ce_train.csv', index=False)
ep_dev.to_csv(f'{save_path}/ce_dev.csv', index=False)

p_binary_train.to_csv(f'{save_path}/pc_binary_train.csv', index=False)
p_binary_dev.to_csv(f'{save_path}/pc_binary_dev.csv', index=False)

### length test

In [ ]:
ep_train, ep_dev, p_binary_train, p_binary_dev

In [26]:
DATA_V = 'uncleaned_v3'
model_checkpoint = 'snunlp/KR-ELECTRA-discriminator'

train_path = f'./dataset/{DATA_V}/raw_train.csv'
dev_path = f'./dataset/{DATA_V}/raw_dev.csv'
test_path = f'./dataset/{DATA_V}/raw_test.csv'
train = pd.read_csv(train_path)
dev = pd.read_csv(dev_path)
test = pd.read_csv(test_path)

### new
entity_property_pair = [
    '본품#가격', '본품#다양성', '본품#디자인', '본품#인지도', '본품#일반', '본품#편의성', '본품#품질',
    '브랜드#가격', '브랜드#디자인', '브랜드#인지도', '브랜드#일반', '브랜드#품질',
    '제품 전체#가격', '제품 전체#다양성', '제품 전체#디자인', '제품 전체#인지도', '제품 전체#일반', '제품 전체#편의성', '제품 전체#품질',
    '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#디자인', '패키지/구성품#일반', '패키지/구성품#편의성', '패키지/구성품#품질'
]
special_tokens = ['&name&', '&affiliation&', '&social-security-num&', '&tel-num&', '&card-num&', '&bank-account&', '&num&', '&online-account&']
emojis = pd.concat([train.sentence_form, dev.sentence_form, test.sentence_form], ignore_index=True, verify_integrity=True).to_frame()
emojis = list(set(demoji.findall(' '.join(emojis.sentence_form.to_list())).keys()))
ep_labels = pd.Series(entity_property_pair, name='sentence_form', copy=True)

tokens2add = special_tokens + emojis

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
print(len(tokenizer))
tokenizer_train_data = pd.concat([train.sentence_form, dev.sentence_form, test.sentence_form], ignore_index=True, verify_integrity=True).to_frame().drop_duplicates()
tokenizer_train_data = tokenizer_train_data.sentence_form.to_list()
new_tokenizer = tokenizer.train_new_from_iterator(tokenizer_train_data, vocab_size=1)
new_tokens = set(list(new_tokenizer.vocab.keys()) + tokens2add) - set(tokenizer.vocab.keys())
tokenizer.add_tokens(list(new_tokens))
print(len(new_tokenizer))
print(len(tokenizer))
# model.resize_token_embeddings(len(tokenizer))

30000



3060
30117


In [29]:
ep_train, ep_dev, p_binary_train, p_binary_dev
len_counter = []
for df in [ep_train, ep_dev, p_binary_train, p_binary_dev]:
    for idx, row in df.iterrows():
        len_counter.append(len(tokenizer(row["form"], row["pair"], truncation=True).input_ids))

In [30]:
max(len_counter)

138

### done here.

In [ ]:
ep_total = pd.concat([ep_train, ep_dev])
p_total = pd.concat([p_train, p_dev])
len(ep_total), len(p_total)

(144825, 6198)

In [ ]:
ep_total.reset_index(inplace=True, drop=True)
p_total.reset_index(inplace=True, drop=True)

In [ ]:
p_total_binary = reformat_p_binary(p_total)
p_total_binary = pd.DataFrame(p_total_binary, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [ ]:
p_total_binary.sort_values(['id', 'entity_property', 'labels'], inplace=True, ascending=[True, False, True])

In [ ]:
p_dev_binary = reformat_p_binary(p_dev)
p_dev_binary = pd.DataFrame(p_dev_binary, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [ ]:
p_dev_binary.sort_values(['id', 'entity_property', 'labels'], inplace=True, ascending=[True, False, True])

In [ ]:
test

,id,sentence_form,annotation
0,nikluge-sa-2022-test-00001,하나 사려고 알아보는 중인데 맘에드는거 발견,[]
1,nikluge-sa-2022-test-00002,동양인 피부톤과 잘 어울리고 우아한 분위기를 풍긴다네,[]
2,nikluge-sa-2022-test-00003,근데 이건 마르살라보다 더 지나친 색 같은데,[]
3,nikluge-sa-2022-test-00004,나스 색조가 다 그렇지만서도 어데이셔스 라인은 진짜 색 기막히게 뽑는것 같다,[]
4,nikluge-sa-2022-test-00005,색상만 보면 이걸 어떻게 발라 싶겠지만 의외로 너무너무 괜찮다,[]
...,...,...,...
2122,nikluge-sa-2022-test-02123,간단한 충전으로 간편한 사용이 가능한거죠,[]
2123,nikluge-sa-2022-test-02124,눈을 가린 상태에서도 간편하게 조작이 가능하구요,[]
2124,nikluge-sa-2022-test-02125,다양한 마사지로 관자놀이부터 눈주변까지 부드럽고 강력한 마사지가 진행됩니다,[]
2125,nikluge-sa-2022-test-02126,본체부터 케이블 설명서까지 깔끔하게 정리되어 보관이 가능하니 더더 맘에 쏙 들어요,[]


In [ ]:
save_path = './dataset/cleaned_v2_total_v1'
!mkdir -p {save_path}

train.to_csv(f'{save_path}/raw_train.csv', index=False)
dev.to_csv(f'{save_path}/raw_dev.csv', index=False)
test.to_csv(f'{save_path}/raw_test.csv', index=False)

ep_total.to_csv(f'{save_path}/ce_train.csv', index=False)
ep_dev.to_csv(f'{save_path}/ce_dev.csv', index=False)

p_total_binary.to_csv(f'{save_path}/pc_binary_train.csv', index=False)
p_dev_binary.to_csv(f'{save_path}/pc_binary_dev.csv', index=False)

## Counting

In [ ]:
print('binary_multi: ', end=''), print(len(ep_train), len(ep_dev), len(p_train), len(p_dev))
print('binary_binary: ', end=''), print(len(ep_train), len(ep_dev), len(p_binary_train), len(p_binary_dev))
ep_train = ep_train.drop_duplicates()
ep_dev = ep_dev.drop_duplicates()
p_train = p_train.drop_duplicates()
p_dev = p_dev.drop_duplicates()
p_binary_train = p_binary_train.drop_duplicates()
p_binary_dev = p_binary_dev.drop_duplicates()
print('\nafter drop_duplicates')
print('binary_multi: ', end=''), print(len(ep_train), len(ep_dev), len(p_train), len(p_dev))
print('binary_binary: ', end=''), print(len(ep_train), len(ep_dev), len(p_binary_train), len(p_binary_dev))

## Save Files

In [ ]:
save_path = './dataset/cleaned_v1'

train.to_csv(f'{save_path}/raw_train.csv', index=False)
dev.to_csv(f'{save_path}/raw_dev.csv', index=False)
test.to_csv(f'{save_path}/raw_test.csv', index=False)

ep_train.to_csv(f'{save_path}/ce_train.csv', index=False)
ep_dev.to_csv(f'{save_path}/ce_dev.csv', index=False)
p_train.to_csv(f'{save_path}/pc_train.csv', index=False)
p_dev.to_csv(f'{save_path}/pc_dev.csv', index=False)
p_binary_train.to_csv(f'{save_path}/pc_binary_train.csv', index=False)
p_binary_dev.to_csv(f'{save_path}/pc_binary_dev.csv', index=False)

# ASC Augmentation

In [ ]:
model_checkpoint = '/content/drive/MyDrive/aspect_based_sentiment_analysis/base_model/klue_roberta_base/v2/klue_roberta_base_mlm/checkpoint-19860'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [ ]:
sTokens = tokenizer.all_special_tokens

def delTokens(sent):
    sent = sent.split(' ')
    temp = []
    for e in sent:
        if e not in sTokens:
            temp.append(e)
    return ' '.join(temp)

In [ ]:
positive, negative, neutral = p_train[p_train.sentiment == 'positive'], p_train[p_train.sentiment == 'negative'], p_train[p_train.sentiment == 'neutral']

In [ ]:
len(positive), len(negative), len(neutral)

In [ ]:
(58 * 3) * 4 * 3, (95 * 3) * 4 * 2 # bt ri rr

Back Translation / Random Insertion / Random Replacement / Random Swap / Random Deletion

In [ ]:
def backTrans(text):
    aug1 = ts.papago(text, sleep_seconds=5, from_language='ko', to_language='en')
    aug1 = ts.papago(aug1, sleep_seconds=5, from_language='en', to_language='ko')

    aug2 = ts.papago(text, sleep_seconds=5, from_language='ko', to_language='ja')
    aug2 = ts.papago(aug2, sleep_seconds=5, from_language='ja', to_language='ko')

    return [aug1, aug2]

def randomInsert(num, sample, device):
    aug = naw.ContextualWordEmbsAug(
        model_path=model_checkpoint, action="insert", model_type='bert', top_k=5, aug_p=0.3, aug_min=1, aug_max=1, device=device)

    aug_result = aug.augment(sample, n=num, num_thread=12)
    aug_result = list(map(delTokens, aug_result))
    aug_result = list(set(aug_result))
    return aug_result

def randomReplace(num, sample, device):
    aug = naw.ContextualWordEmbsAug(
        model_path=model_checkpoint, action="insert", model_type='bert', top_k=5, aug_p=0.3, aug_min=1, aug_max=1, device=device)

    aug_result = aug.augment(sample, n=num, num_thread=12)
    aug_result = list(map(delTokens, aug_result))
    aug_result = list(set(aug_result))
    return aug_result

def randomSwap(num, sample):
    aug = naw.RandomWordAug(action='swap', aug_min=1, aug_max=1, aug_p=0.3)    
    aug_result = aug.augment(sample, n=num, num_thread=2)
    aug_result = list(set(aug_result))
    return aug_result

def randomSplit(num, sample):
    aug = naw.SplitAug(aug_min=1, aug_max=1, aug_p=0.3, min_char=3)
    aug_result = aug.augment(sample, n=num, num_thread=2)
    aug_result = list(set(aug_result))
    return aug_result

In [ ]:
(58 * 3) * 5 * 4, (95 * 3) * 4 * 3 # bt ri rr

In [ ]:
def backtransRoutine(data2augment, output_path):
    print('back translation started.')
    temp = []
    for row in data2augment:
        augs = backTrans(row[1])
        for aug in augs:
            if aug != '' and aug != row[1]:
                new = [row[0], aug, row[2], row[3]]
                if new not in data2augment:
                    temp.append(new)
    data2augment.extend(temp)
    print(len(f'back translation finished.\ncurrent count: {len(data2augment)}'))

    data_aug = pd.DataFrame(data2augment, columns=['id', 'sentence_form', 'entity_property', 'sentiment'])
    data_aug.to_csv(f'{output_path}', index=False)

    return data_aug

In [ ]:
import os

def edaRoutine(data2augment, ri, rr, output_path):
    print(f'current count: {len(data2augment)}')
    print('random insertion started.')
    temp = []
    for row in data2augment:
        augs = randomInsert(ri, row[1], 'cuda')
        for aug in augs:
            if aug != '' and aug != row[1]:
                new = [row[0], aug, row[2], row[3]]
                if new not in data2augment:
                    temp.append(new)
    data2augment.extend(temp)
    print(f'random insertion finished.\ncurrent count: {len(data2augment)}')

    print('random replacement started.')
    temp = []
    for row in data2augment:
        augs = randomReplace(rr, row[1], 'cuda')
        for aug in augs:
            if aug != '' and aug != row[1]:
                new = [row[0], aug, row[2], row[3]]
                if new not in data2augment:
                    temp.append(new)
    data2augment.extend(temp)
    print(f'random replacement finished.\ncurrent count: {len(data2augment)}')

    print('random swap and split started.')
    while len(data2augment) < len(positive):
        temp = []
        k = random.randrange(len(negative))
        id, text, entity, sentiment = data2augment[k]

        selector = random.randint(0,1)
        if selector == 0:
            augs = randomSwap(1, text)
            for aug in augs:
                if aug != '' and aug != text:
                    new = [id, aug, entity, sentiment]
                    if new not in data2augment:
                        temp.append(new)
            data2augment.extend(temp)
        else:
            augs = randomSplit(1, text)
            for aug in augs:
                if aug != '' and aug != text:
                    new = [id, aug, entity, sentiment]
                    if new not in data2augment:
                        temp.append(new)
            data2augment.extend(temp)
        if len(data2augment)%25 == 0:
            print(f'random swap and split in progress.\ncurrent count: {len(data2augment)}')

    print(f'whole augmentation routine finished.\ntotal count: {len(data2augment)}')

    data_aug = pd.DataFrame(data2augment, columns=['id', 'sentence_form', 'entity_property', 'sentiment'])
    data_aug.to_csv(f'{output_path}', index=False)

    return data_aug

In [ ]:
### negative
# # back translation

# data2augment = negative.values.tolist()

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'negative_bt.csv'
output_path = os.path.join(output_folder, output_file)

# negative_bt = backtransRoutine(data2augment, output_path)
negative_bt = pd.read_csv(output_path)
negative_bt = negative_bt.values.tolist()
# RI / RR

ri = 4 # times - 1
rr = 3 # times - 1

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'negative_aug.csv'
output_path = os.path.join(output_folder, output_file)

# negative_aug = edaRoutine(negative_bt, ri, rr, output_path)
negative_aug = pd.read_csv(output_path)

In [ ]:
negative_aug
negative_aug = negative_aug.drop_duplicates()

In [ ]:
# negative_aug.sample(n=15, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False)

In [ ]:
# negative_aug.sort_values('id').head(50).sentence_form

In [ ]:
### neutral
# back translation

# data2augment = neutral.values.tolist()

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'neutral_bt.csv'
output_path = os.path.join(output_folder, output_file)

# neutral_bt = backtransRoutine(data2augment, output_path)
neutral_bt = pd.read_csv(output_path)
neutral_bt = neutral_bt.values.tolist()

# RI / RR

ri = 3 # times - 1
rr = 2 # times - 1

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'neutral_aug.csv'
output_path = os.path.join(output_folder, output_file)

# neutral_aug = edaRoutine(neutral_bt, ri, rr, output_path)
neutral_aug = pd.read_csv(output_path)

In [ ]:
neutral_aug
neutral_aug = neutral_aug.drop_duplicates()

In [ ]:
# neutral_aug.sample(n=15, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False)

In [ ]:
# neutral_aug.sort_values('id').head(50).sentence_form

In [ ]:
p_train_aug = pd.concat([positive, negative_aug, neutral_aug])

In [ ]:
def reformat_p_binary(df):
    p_binary = []
    for i, row in df.iterrows():
        row.id, row.sentence_form, row.entity_property, row.sentiment
        for sentiment in polarity_id_to_name:
            if sentiment == row.sentiment:
                p_binary.append([row.id, row.sentence_form, '#'.join([row.entity_property, row.sentiment]), tf_name_to_id['True']])
            else: 
                p_binary.append([row.id, row.sentence_form, '#'.join([row.entity_property, sentiment]), tf_name_to_id['False']])
    return p_binary

In [ ]:
p_binary_train_aug = reformat_p_binary(p_train_aug)
p_binary_train_aug = pd.DataFrame(p_binary_train_aug, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [ ]:
p_binary_train_aug

In [ ]:
p_binary_dev = reformat_p_binary(p_dev)
p_binary_dev = pd.DataFrame(p_binary_dev, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [ ]:
p_binary_dev

# Counting

In [ ]:
len(ep_train), len(ep_dev), len(p_train), len(p_dev)

In [ ]:
len(ep_train), len(ep_dev), len(p_binary_train_aug), len(p_binary_dev)

In [ ]:
ep_train = ep_train.drop_duplicates()
p_binary_train_aug = p_binary_train_aug.drop_duplicates()
ep_dev = ep_dev.drop_duplicates()
p_binary_dev = p_binary_dev.drop_duplicates()
len(ep_train), len(ep_dev), len(p_binary_train_aug), len(p_binary_dev)

# Export

In [ ]:
%cd /content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11

# train.to_csv('raw_train.csv', index=False)
# dev.to_csv('raw_dev.csv', index=False)
# test.to_csv('raw_test.csv', index=False)

ep_train.to_csv('ce_train.csv', index=False)
p_binary_train_aug.to_csv('pc_binary_train_aug.csv', index=False)
ep_dev.to_csv('ce_dev.csv', index=False)
p_binary_dev.to_csv('pc_binary_dev.csv', index=False)

In [ ]:
# emojis = pd.concat([ep_train.sentence_form, p_train.sentence_form, ep_dev.sentence_form, p_dev.sentence_form], ignore_index=True, verify_integrity=True).to_frame()
# emojis = list(set(demoji.findall(' '.join(emojis.sentence_form.to_list())).keys()))

In [ ]:
df = pd.read_csv('ce_train.csv')
df[df.id == 'nikluge-sa-2022-train-00065']

In [ ]:
df = pd.read_csv('ce_dev.csv')
df

In [ ]:
df = pd.read_csv('pc_binary_train_aug.csv')
df

In [ ]:
df = pd.read_csv('pc_binary_dev.csv')
df